# RSA accumulator proof using Bézout coefficients


In [47]:
!cat /proc/cpuinfo | grep 'model name'

model name	: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz
model name	: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz
model name	: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz
model name	: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz
model name	: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz
model name	: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz
model name	: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz
model name	: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz


In [48]:
%%capture
!pip install pycryptodome

In [49]:
from Crypto.Util import number
import numpy as np
import math

In [50]:
def bezout(x, u):
    if u == 0:
        return (x, 1, 0)
    else:
        g, x1, y1 = bezout(u, x % u)
        return (g, y1, x1 - (x // u) * y1)

## Math:
Remember from math chapter the proof:

$g^{ax+bu}  \pmod N \equiv g \pmod N$

were:

$ N=p*q $

$ {\phi(N)}=(p−1)(q−1) $

$A=g^u \pmod N $

$u = \prod_0^{k-1} x_i$

## # RSA accumulator

In [51]:
p, q = number.getPrime(32), number.getPrime(32) # note that we do not need secure
n = p*q
g = 3

# The final accumulator value
u = number.getRandomNBitInteger(65) # dummy value for revoked attestations

# The attestation to do exclusion proof with
x = number.getPrime(30) # the attestation unique id

## Proof

$g^{ax+ub} = g^1 \pmod N$

In [52]:
# First we check the GCD(x,u)
# We then get the bezout coefficients

gcd, a, b = bezout(x, u)

assert gcd == 1

# We are now ready to compute the non membership proof
proof = pow(g, a*x + b*u, n)

# The proof must be equal to the generator
print(f'The Bezout proof is valid: {proof == g}')

The Bezout proof is valid: True


## Proof

$u' = u \pmod {\phi(N)}$

$g^{ax+ub} = g^{ax + bu'} = g^1 \pmod N$

In [53]:
phi = (p-1)*(q-1)
u_prim = u % phi

# We are now ready to compute the non membership proof
proof = pow(g, a*x + b*u_prim, n)

print(f'The Bezout proof is valid: {proof == g}')

The Bezout proof is valid: True


## Large proof

In [54]:
# accumulator
u = 1
number_primes = 2000

primes = []
for i in range(number_primes):
  tmp = number.getPrime(30)
  primes.append(tmp)
  u = u * tmp

In [55]:
gcd, a, b = bezout(x, u)

assert gcd == 1

proof = pow(g, a*x + b*u, n)
print('The Bezout proof is valid:', proof == g)

# Checking that totient optimization works too
proof = pow(g, (a % phi)*x+b*(u % phi), n)
print('The Bezout proof is valid:', proof == g)

print(f'The issuer publishes {u % phi} and includes {a % phi} in the attestation')

The Bezout proof is valid: True
The Bezout proof is valid: True
The issuer publishes 2446094601988303339 and includes 7422031670695948280 in the attestation


## Exclusion proof

In [56]:
x = primes[0] # this value should return False
gcd, a, b = bezout(x, u)

proof = pow(g, a*x + b*u, n)
print('The Bezout proof is valid:', proof == g)

# Checking that totient optimization works too
proof = pow(g, (a % phi)*x+b*(u % phi), n)
print('The Bezout proof is valid:', proof == g)

The Bezout proof is valid: False
The Bezout proof is valid: False


In [57]:
x = number.getPrime(30) # this value should return True
gcd, a, b = bezout(x, u)

proof = pow(g, a*x + b*u, n)
print('The Bezout proof is valid:', proof == g)

# Checking that totient optimization works too
proof = pow(g, (a % phi)*x+b*(u % phi), n)
print('The Bezout proof is valid:', proof == g)

The Bezout proof is valid: True
The Bezout proof is valid: True


In [58]:
hex(a % phi), hex(u % phi)

('0x758923e6e95c7a1a', '0x21f246149e8369eb')

## How to calculate the new accumulator value after update
Accumulator is calculted as:

$A=g^u \pmod N = g^{\prod_0^{k-1} x_i} \pmod N$

### Add
$A' = A^{x_i} \pmod N$

### Delete

If $ {\phi(N)} $ is known it's simplier to calculate:

$A' = A^{x_i^{-1} \pmod {\phi(N)}}$

In [64]:
x = primes[0] # this value should return False
gcd, a, b = bezout(x, u)

proof = pow(g, a*x + b*u, n)
print('The Bezout proof is valid:', proof == g)

# Checking that totient optimization works too
proof = pow(g, (a % phi)*x+b*(u % phi), n)
print('The Bezout proof is valid:', proof == g)

The Bezout proof is valid: False
The Bezout proof is valid: False


In [65]:
# delete c from accumulator
# the value should return True
A = g
x_inv = pow(x, -1, phi)
A_prim = pow(A, x_inv, n)

gcd, a, b = bezout(x, u)

proof = pow(A_prim, a*x + b*u, n)
print('The Bezout proof is valid:', proof == g)

# Checking that totient optimization works too
proof = pow(A_prim, (a % phi)*x+b*(u % phi), n)
print('The Bezout proof is valid:', proof == g)

The Bezout proof is valid: True
The Bezout proof is valid: True


In [66]:
x = primes[1] # this value should return False
gcd, a, b = bezout(x, u)

proof = pow(A_prim, a*x + b*u, n)
print('The Bezout proof is valid:', proof == g)

# Checking that totient optimization works too
proof = pow(A_prim, (a % phi)*x+b*(u % phi), n)
print('The Bezout proof is valid:', proof == g)

The Bezout proof is valid: False
The Bezout proof is valid: False
